In [1]:
# !pip install kfp==1.0.1 --user
import kfp
from kubernetes.client.models import V1EnvVar

In [2]:
@kfp.dsl.component
def my_component1():
  ...
  return kfp.dsl.ContainerOp(
    name='component1',
    image='mnistclassify:2.0'
  )

In [3]:
@kfp.dsl.component
def my_component2():
  ...
  return kfp.dsl.ContainerOp(
    name='component2',
    image='mnistpredict:2.0'
  )

In [4]:
#Defining the pipeline that uses the above components.
@kfp.dsl.pipeline(name='pipeline2', description='mnist tf model')
def model_generation():
    component1 = (my_component1()
                  .add_env_variable(V1EnvVar(name = 'ip', value = 'X.X.X.X')) 
                  .add_env_variable(V1EnvVar(name = 'user', value = 'root' )) 
                  .add_env_variable(V1EnvVar(name = 'passwd', value = 'admin123' ))
    )

    component1.container.set_image_pull_policy("IfNotPresent")
    
    component2 = my_component2().after(component1)
    component2.container.set_image_pull_policy("IfNotPresent")


In [5]:
pipeline_func = model_generation
pipeline_filename = pipeline_func.__name__ + '.pipeline2.zip'

#Compile the pipeline and export it
kfp.compiler.Compiler().compile(model_generation,
                           pipeline_filename)